In [5]:
from bs4 import BeautifulSoup
import requests
from itertools import chain
import pandas as pd
from tqdm import tqdm_notebook

In [3]:
def create_link(year):
    return f"https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id={year};trophy=89;type=season"

In [7]:
links = [create_link(yr) for yr in ["2007/08", "2009", "2010", "2012/13", "2013/14", "2015/16"]]

In [15]:

country_id = {
    "6": "India",
    "26": "Kenya",
    "2": "Australia",
    "25": "Bangladesh",
    "1": "England",
    "29": "Ireland",
    "15": "Netherlands",
    "5": "NewZealand",
    "7": "Pakistan",
    "30": "Scotland",
    "3": "SouthAfrica",
    "8": "SriLanka",
    "4": "WestIndies",
    "9": "Zimbabwe",
    "17": "Canada"
    
    
}

In [16]:
country_links = [
    "https://stats.espncricinfo.com/ci/engine/records/averages/[replace].html?id=12;team=6;type=trophy",
    "https://stats.espncricinfo.com/ci/engine/records/averages/[replace].html?id=12;team=26;type=trophy",
    "https://stats.espncricinfo.com/ci/engine/records/averages/[replace].html?id=12;team=2;type=trophy",
    "https://stats.espncricinfo.com/ci/engine/records/averages/[replace].html?id=12;team=25;type=trophy",
    "https://stats.espncricinfo.com/ci/engine/records/averages/[replace].html?id=12;team=1;type=trophy",
    "https://stats.espncricinfo.com/ci/engine/records/averages/[replace].html?id=12;team=29;type=trophy",
    "https://stats.espncricinfo.com/ci/engine/records/averages/[replace].html?id=12;team=15;type=trophy",
    "https://stats.espncricinfo.com/ci/engine/records/averages/[replace].html?id=12;team=5;type=trophy",
    "https://stats.espncricinfo.com/ci/engine/records/averages/[replace].html?id=12;team=7;type=trophy",
    "https://stats.espncricinfo.com/ci/engine/records/averages/[replace].html?id=12;team=30;type=trophy",
    "https://stats.espncricinfo.com/ci/engine/records/averages/[replace].html?id=12;team=3;type=trophy",
    "https://stats.espncricinfo.com/ci/engine/records/averages/[replace].html?id=12;team=8;type=trophy",
    "https://stats.espncricinfo.com/ci/engine/records/averages/[replace].html?id=12;team=4;type=trophy",
    "https://stats.espncricinfo.com/ci/engine/records/averages/[replace].html?id=12;team=9;type=trophy",
    "https://stats.espncricinfo.com/ci/engine/records/averages/[replace].html?id=12;team=17;type=trophy"  

]

In [17]:
def scrape_page(url, country):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find_all("table", {"class":"engineTable"})[0]
    columns = [col.text.strip() for col in soup.find("tr", {"class": "head"}).find_all("th")]
    all_rows = []
    for row in table.find_all("tr")[1:]:
        elements = [el.text for el in row.find_all("td")]
        all_rows.append(elements)
    df = pd.DataFrame(all_rows, columns=columns)
    df["country"] = country_id[country]
    return df

In [18]:
def scrape_stats(stat_type):
    links = [link.replace("[replace]", stat_type) for link in country_links]
    all_df = []
    for link in tqdm_notebook(links):
        c_id = link.split("?")[1].split(";")[1].split("=")[1]
        all_df.append(scrape_page(link, c_id))
    return pd.concat(all_df)

In [19]:
batting_df = scrape_stats("batting")

/home/mancunian92/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


HBox(children=(FloatProgress(value=0.0, max=15.0), HTML(value='')))

In [20]:
bowling_df = scrape_stats("bowling")

/home/mancunian92/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


HBox(children=(FloatProgress(value=0.0, max=15.0), HTML(value='')))

In [21]:
batting_df.to_csv("./worldcup_data/all_players_batting_stats.csv", index=False)
bowling_df.to_csv("./worldcup_data/all_players_bowling_stats.csv", index=False)

In [ ]:
## Individual records
